# JLA Supernova sample

JLA data taken from [here](http://supernovae.in2p3.fr/sdss_snls_jla/ReadMe.html) (*jla_light_curves.tgz* file). For more information about the calibration of the sample check [Betoule et al. (2014)](https://ui.adsabs.harvard.edu/#abs/arXiv:1401.4064). Six SNe (sn2005hc, sn2005ir, sn2006nz, sn2006oa, sn2006ob, sn2006on) are in common with SDSS and either CfAIII or CSP (therefore the measurements are in different magnitude systems). The light-curve files contains a merge of all available photometry for these SNe.

Different filters used taken from _jla-instruments-2-4.tar.gz_ file in [here](http://supernovae.in2p3.fr/salt/doku.php?id=instruments). Filters corresponding to the different surveys/samples are listed below:

CalanTololo $\rightarrow$ STANDARD (SNLS3-Landolt-model)

low-z $\rightarrow$ STANDARD (SNLS3-Landolt-model)

CSP $\rightarrow$ SWOPE2

CfAI $\rightarrow$ STANDARD (SNLS3-Landolt-model)

CfAII $\rightarrow$ STANDARD (SNLS3-Landolt-model)

CfAIII $\rightarrow$ KEPLERCAM & 4SHOOTER2

SNLS $\rightarrow$ MEGACAMPSF (Megacam-PSF; SNLS3-Megacam-model)

SDSS $\rightarrow$ SDSS

__See the 'fitmodel.card' file for more info regarding the magnitude systems and offsets__

In [1]:
import pandas as pd
import numpy as np
import glob
import os

jla_lcparams = pd.read_csv('test_files/JLA/jla_lcparams.txt', delim_whitespace=True)
lc_files = [file for file in glob.glob('test_files/JLA/jla_light_curves/lc*')]
#lc_files = [file for file in lc_files if 'lc-sn' in file]

surveys_count = {'CalanTololo':0, 'lowz':0, 'CSP':0, 'CfAI':0, 'CfAII':0, 'CfAIII':0, 'SNLS':0, 'SDSS':0, 'HST':0}
count_sn_per_survey = True

for file in lc_files:

    sn_df = pd.read_csv(file, delim_whitespace=True, comment='@', names=['mjd', 'flux', 'flux_err', 'zp', 'band', 'mag_sys']).iloc[7:]
    sn = file.split('lc-')[-1][:-5]
    
    # z, ra, dec
    sn_info = pd.read_csv('test_files/JLA/jla_lcparams.txt', delim_whitespace=True)
    sn_info = sn_info[sn_info['#name']==sn]
    z = sn_info.zcmb.values[0]
    ra = sn_info.ra.values[0]
    dec = sn_info.dec.values[0]

    if count_sn_per_survey:
        with open(file) as search:
            for line in search:
                if '@SURVEY' in line:
                    survey_name = line.split()[-1]
                    surveys_count[survey_name] += 1
                    break
                if '@SET' in line and 'RiessHST' in line:
                    surveys_count['HST'] += 1
                    break
    
    # rename bands
    sn_df['band'] = sn_df['band'].str.replace('STANDARD::', 'Standard_') 
    sn_df['band'] = sn_df['band'].str.replace('SWOPE2::', 'csp_') 
    sn_df['band'] = sn_df['band'].str.replace('KEPLERCAM::', 'Keplercam_') 
    sn_df['band'] = sn_df['band'].str.replace('4SHOOTER2::', '4Shooter2_') 
    sn_df['band'] = sn_df['band'].str.replace('MEGACAMPSF::', 'Megacam_')
    sn_df['band'] = sn_df['band'].str.replace('SDSS::', 'sdss_') 
    sn_df['band'] = sn_df['band'].str.replace('ACSWF::', 'ACSWF_') 
    sn_df['band'] = sn_df['band'].str.replace('NICMOS2::', 'NICMOS2_') 
    
    # rename magnitude system
    sn_df['mag_sys'] = sn_df['mag_sys'].str.replace('VEGA2', 'BD17')
    sn_df['mag_sys'] = sn_df['mag_sys'].str.replace('VEGAHST', 'BD17')
    sn_df['mag_sys'] = sn_df['mag_sys'].str.replace('AB_B12', 'AB')
    
    sn_df[['flux', 'flux_err']] = sn_df[['flux', 'flux_err']].round(3)
    
    sn_file_name = f'data_jla/{sn}.dat'
    with open(sn_file_name, 'w') as file:
        file.write('name z ra dec\n')
        file.write(f'{sn} {z} {ra} {dec} \n')
    sn_df[['mjd', 'flux', 'flux_err', 'zp', 'band', 'mag_sys']].to_csv(sn_file_name, index=False, sep=' ', mode='a')
        
if count_sn_per_survey:
    print(surveys_count)
    n = [val for val in surveys_count.values()]
    print(f'Total of {np.sum(n)} SNe')
print('ready')

{'CalanTololo': 17, 'lowz': 11, 'CSP': 13, 'CfAI': 7, 'CfAII': 15, 'CfAIII': 55, 'SNLS': 239, 'SDSS': 374, 'HST': 9}
Total of 740 SNe
ready
